In [ ]:
import pandas as pd
import numpy as np 
from torch.utils.data import TensorDataset, DataLoader
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
import torch
import random
import numpy as np

# Set a fixed seed
seed = 1
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

# Ensure deterministic behavior
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


In [ ]:
xiang_filtered = pd.read_csv("xiang_filtered.csv")
xiang_filtered_embeddings = torch.load("filtered_embeddings.pt")


In [ ]:
ccad_filtered = pd.read_csv("clustercad_filtered.csv")
ccad_filtered_embeddings = torch.load("filtered_ccad_embeddings.pt")

In [ ]:
print(len(ccad_filtered))
print(len(ccad_filtered_embeddings))

In [ ]:
'''
to_add = ["C","C1", "C2", "B1", "B2", "A2"]
ccad_filtered = ccad_filtered[ccad_filtered["kr_type_annotation"].isin(to_add)]
'''

to_add = ["C","C2","C1","B", "B2","A","A1", "A2","B1"]
ccad_filtered = ccad_filtered[ccad_filtered["kr_type_annotation"].isin(to_add)]

In [ ]:
ccad_filtered["kr_type_annotation"].unique()

In [ ]:
ccad_filtered[ccad_filtered["kr_type_annotation"] == "A"]

In [ ]:
ccad_filtered

In [ ]:
#cut 600 b1

# Identify all B1 rows
b1_mask = ccad_filtered["kr_type_annotation"] == "B1"
b1_indices = ccad_filtered[b1_mask].index

# Randomly sample 600 B1 indices to drop
b1_to_remove = np.random.choice(b1_indices, size=700, replace=False)

# Drop those rows from the DataFrame
ccad_filtered = ccad_filtered.drop(index=b1_to_remove)


#cut 100 a1

# Identify all B1 rows
a1_mask = ccad_filtered["kr_type_annotation"] == "A1"
a1_indices = ccad_filtered[a1_mask].index

# Randomly sample 600 B1 indices to drop
a1_to_remove = np.random.choice(a1_indices, size=104, replace=False)

# Drop those rows from the DataFrame
ccad_filtered = ccad_filtered.drop(index=a1_to_remove)


In [ ]:
ccad_filtered

In [ ]:
ccad_filtered

In [ ]:
xiang_idx = xiang_filtered.index.to_numpy().tolist()
xiang_filtered_embeddings = [xiang_filtered_embeddings[i] for i in xiang_idx]
print(len(xiang_filtered_embeddings))

In [ ]:
ccad_filtered

In [ ]:
clustercad_idx = ccad_filtered.index.to_numpy().tolist()
ccad_filtered_embeddings = [ccad_filtered_embeddings[i] for i in clustercad_idx]
print(len(ccad_filtered_embeddings))

In [ ]:
ccad_filtered

In [ ]:
ccad_filtered = ccad_filtered.reset_index()
xiang_filtered = xiang_filtered.reset_index()

In [ ]:
print(len(ccad_filtered))
print(len(ccad_filtered_embeddings))

In [ ]:
xiang_filtered["Annotation"] == "B"

In [ ]:
sum(ccad_filtered["kr_type_annotation"] == "B")

In [ ]:

#cut 30 b1

# Identify all B rows
b_mask = xiang_filtered["Annotation"] == "B"
b_indices = xiang_filtered[b_mask].index

# Randomly sample 30 B indices to drop
b_to_remove = np.random.choice(b_indices, size=10, replace=False)

# Drop those rows from the DataFrame
xiang_filtered = xiang_filtered.drop(index=b_to_remove)

In [ ]:
#xiang_idx

In [ ]:
len(xiang_filtered_embeddings)

In [ ]:

xiang_idx = xiang_filtered.index.to_numpy().tolist()
xiang_filtered_embeddings = [xiang_filtered_embeddings[i] for i in xiang_idx]
print(len(xiang_filtered_embeddings))


In [ ]:
print(len(xiang_filtered[xiang_filtered["Annotation"] == "B1"]))
print(len(ccad_filtered[ccad_filtered["kr_type_annotation"] == "B1"]))


In [ ]:
#average pooling. 
xiang_filtered_embeddings = [x.mean(dim=1).squeeze(0) for x in xiang_filtered_embeddings]
ccad_filtered_embeddings = [x.mean(dim=1).squeeze(0) for x in ccad_filtered_embeddings]

In [ ]:
print(xiang_filtered_embeddings[0].shape)
print(ccad_filtered_embeddings[0].shape)


In [ ]:
len(xiang_filtered_embeddings)

In [ ]:
#stacking
xiang_embeddings = torch.stack(xiang_filtered_embeddings)
ccad_embeddings = torch.stack(ccad_filtered_embeddings)

In [ ]:
#outputs need to be converted to numerical values.

annotations_unique = ccad_filtered["kr_type_annotation"].unique()
annotations_unique.sort() #sort alphabetically 

annotation_enumerated = {x: i for i, x in enumerate(annotations_unique)}
print(annotation_enumerated)

'''
annotations_unique_cc = ccad_filtered["kr_type_annotation"].unique()
annotations_unique_cc.sort() #sort alphabetically 

annotation_enumerated_cc = {x: i for i, x in enumerate(annotations_unique_cc)}
print(annotation_enumerated_cc)
'''

In [ ]:
print(xiang_filtered["Annotation"])

In [ ]:
ccad_filtered

In [ ]:
#xiang_filtered["AnnotationEnumerated"] = xiang_filtered["Annotation"].map(annotation_enumerated)
ccad_filtered["AnnotationEnumerated"] = ccad_filtered["kr_type_annotation"].map(annotation_enumerated)
#ccad_filtered['AnnotationEnumerated'] = ccad_filtered['AnnotationEnumerated'].astype(np.int64)

In [ ]:
annotation_enumerated

In [ ]:
#print(xiang_filtered["AnnotationEnumerated"])
print(ccad_filtered["AnnotationEnumerated"])

In [ ]:
#xiang_filtered_np = xiang_filtered["AnnotationEnumerated"].to_list()
ccad_filtered_np = ccad_filtered["AnnotationEnumerated"].to_list()

In [ ]:
#ccad_filtered_np

In [ ]:
#xiang_filtered["Annotation"].to_numpy()[5]

In [ ]:
#ccad_filtered["kr_type_annotation"].to_numpy()[929]

In [ ]:
#ccad_filtered_np[929]

In [ ]:
#xiang_filtered_tensor = torch.tensor(xiang_filtered_np)
#xiang_filtered_tensor = [torch.tensor(x, dtype=torch.long) for x in xiang_filtered_np]

#xiang_filtered_tensor = torch.tensor(xiang_filtered_np, dtype=torch.long)
ccad_filtered_tensor = torch.tensor(ccad_filtered_np, dtype=torch.long)
#merged_tensor = torch.cat([xiang_filtered_tensor, ccad_filtered_tensor], dim=0)

In [ ]:
#merged_embeddings = torch.cat([xiang_embeddings, ccad_embeddings], dim=0)

In [ ]:
'''
from sklearn.model_selection import train_test_split

x_train_tensor_merged, x_test_tensor_merged, y_train_tensor_merged, y_test_tensor_merged = train_test_split(
    merged_embeddings,
    merged_tensor,
    test_size=0.2,
    random_state=1,
    stratify=merged_tensor
)

mu_merged, sigma_merged = x_train_tensor_merged.mean(0), x_train_tensor_merged.std(0) + 1e-9

x_train_tensor_merged = (x_train_tensor_merged - mu_merged) / sigma_merged
x_test_tensor_merged = (x_test_tensor_merged - mu_merged) / sigma_merged

print("x train len")
print(len(x_train_tensor_merged))
print("y train len")
print(len(y_train_tensor_merged))
'''

In [ ]:
from sklearn.model_selection import train_test_split

x_train_tensor, x_test_tensor, y_train_tensor, y_test_tensor = train_test_split(
    ccad_embeddings,
    ccad_filtered_tensor,
    test_size = 0.3,
    random_state=1,
    stratify=ccad_filtered_tensor
)

mu, sigma = x_train_tensor.mean(0), x_train_tensor.std(0) + 1e-9
x_train_tensor = (x_train_tensor - mu) / sigma
x_test_tensor = (x_test_tensor - mu) / sigma

print("x train len")
print(len(x_train_tensor))
print("y train len")
print(len(y_train_tensor))

In [ ]:
'''
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(0.20),
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.20),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.20),
            nn.Linear(256,128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.20),
'''

In [ ]:
'''
#add dropout_rate. ==
class kr_predict(nn.Module):
    def __init__(self):
        super(kr_predict, self).__init__()
        self.hidden = nn.Sequential(
            nn.Linear(xiang_embeddings[0].shape[1], 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256,128),
            nn.ReLU(),
            nn.Dropout(0.3),
        )
        self.out = nn.Linear(128, 9)
    def forward(self, x):
        x = x.view(x.size(0), -1) # flatten so we're removing 
        x = self.hidden(x)
        x = self.out(x)
        return x      
'''


class kr_predict(nn.Module):
    def __init__(self):
        super(kr_predict, self).__init__()
        self.hidden = nn.Sequential(
            nn.Linear(1536, 512),     # Slightly wider first layer
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(), 
            nn.Dropout(0.25),
            #nn.Linear(256, 128),
            #nn.BatchNorm1d(128),
            #nn.ReLU(),
            #nn.Dropout(0.3)
        )
        self.out = nn.Linear(256, 9)
    def forward(self, x):
        #x = x.view(x.size(0), -1) # flatten so we're removing - modified so its done w/ squeeze
        x = self.hidden(x)
        x = self.out(x)
        return x 

In [ ]:
#xiang_embeddings[0].shape[1]

In [ ]:
model = kr_predict()

In [ ]:
#apple silicon

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
model.to(device)

if torch.backends.mps.is_available():
    print("MPS is available! Using Apple Silicon GPU.")
else:
    print("MPS is not available. CPU Fallback.")

In [ ]:
'''
from collections import Counter
class_counts = Counter(y_train_tensor.numpy())
print("Class distribution:", class_counts)
#holy fucking imbalance.
#total_samples = sum(class_counts)
#class_weights = [total_samples/(len(class_counts)*count) for count in class_counts]
#class_weights = torch.FloatTensor(class_weights)
#print("Class weights:", class_weights)
#loss = nn.CrossEntropyLoss(weight=class_weights.to(device))

loss = nn.CrossEntropyLoss()
adam = optim.Adam(model.parameters(), lr = .00001)

'''
from collections import Counter
class_counts = Counter(y_train_tensor.numpy())
print(annotation_enumerated)
print("Class distribution:", class_counts)

# Fix: Get counts in proper order (class 0, 1, 2, ..., 8) and avoid zero division
total_samples = sum(class_counts.values())  # Sum the actual counts, not keys
class_weights = []
num_classes = 9


'''
for class_idx in range(9):  # Ensure we have weights for all 9 classes
    count = class_counts.get(class_idx, 0)  # Get count, default to 0 if class doesn't exist
    if count == 0:
        weight = total_samples
        print(f"Warning: Class {class_idx} has 0 samples, setting high weight")
    else:
        weight = total_samples / (num_classes * count)
    class_weights.append(weight)

class_weights = torch.FloatTensor(class_weights)
print("Class weights:", class_weights)
'''

class_weights = torch.tensor([
    total_samples / (num_classes * class_counts.get(i, 1)) for i in range(num_classes)
], dtype=torch.float32).to(device)

loss = nn.CrossEntropyLoss(weight=class_weights.to(device))
adam = optim.AdamW(model.parameters(), lr=0.001)
#scheduler = optim.lr_scheduler.CosineAnnealingLR(adam, T_max=50)
scheduler = optim.lr_scheduler.StepLR(adam, step_size=100, gamma=0.5)
#every 200 epochs, cut in half 0.5

In [ ]:
x_train_tensor = x_train_tensor
y_train_tensor = (y_train_tensor).long()

In [ ]:
print(y_train_tensor)

In [ ]:

#use TensorDatset, DataLoader from torch utils

#worked well on gamma 0.5, epoch 550, batch 8

#batch_size = 32
batch_size = 16
train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
#dropping last bc we are using batchnorm, so it needs >1 batch size 

#1500 optimal
for epoch in range(550):
    model.train()
    epoch_loss = 0.0

    for seqs, anns in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        seqs = seqs.to(device)
        anns = anns.to(device)
        output = model(seqs)
        output_loss = loss(output, anns)
        adam.zero_grad()
        output_loss.backward()
        adam.step()
        epoch_loss += output_loss.item() * seqs.size(0) #batch size scaling
    scheduler.step()
    avg_loss = epoch_loss / len(train_dataset)
    print(f"Epoch {epoch+1}: Loss: {avg_loss:.4f}")


In [ ]:
test_dataset = TensorDataset(x_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
#dropping last bc we are using batchnorm, so it needs >1 batch size 

all_predictions = []
all_targets = []
def accuracy():
    model.eval()
    correct = 0
    total = 0


    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs) #model predictions [batch_size, 9]
            values, predicted = torch.max(outputs.data, 1)
            #values has highest score for each sample in batch
            #the predicted part has the classes w/ highest score for each sample
            total += targets.size(0) #add batch size
            correct += (predicted == targets).sum().item()

            #for classification report
            all_predictions.extend(predicted.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())
    return 100 * correct/total

accuracy()

In [ ]:

from sklearn.metrics import classification_report
import numpy as np


class_names = annotation_enumerated.keys()

print(classification_report(all_targets, all_predictions, target_names=class_names))